In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
def load_abs_surfs():
    abs_surfs = pd.read_excel('data/absSurfs.xlsx').set_index("Site")
    abs_surfs = abs_surfs.stack()
    abs_surfs.index.names = ["Site", "CLC"]
    idx = abs_surfs.index
    abs_surfs.index = abs_surfs.index.set_levels([idx.levels[0].to_series(), idx.levels[1].to_series().astype(int)])
    abs_surfs = pd.DataFrame({'surface': abs_surfs})
    return abs_surfs

In [8]:
def load_pesticides():
    pesticides = pd.read_excel('data/pesticides.xlsx').rename(columns={'importName': 'pesticide'})
    pesticides = pesticides.set_index("pesticide")
    pesticides["typeEN"] = pesticides["typeEN"].str.lower().str.replace("and", ",",regex=True).str.replace(" +", "",regex=True)
    pesticides["familyEN"] = pesticides["familyEN"].str.lower().str.replace(" +", ",",regex=True)
    return pesticides

In [5]:
abs_surfs = load_abs_surfs()
abs_surfs.head(10)

surface
Site  CLC              
F0069 111  0.000000e+00
      112  1.064122e+06
      121  3.587087e+05
      122  0.000000e+00
      123  0.000000e+00
      124  0.000000e+00
      131  0.000000e+00
      132  0.000000e+00
      133  0.000000e+00
      141  0.000000e+00

In [9]:
pesticides = load_pesticides()
pesticides.head(10)

,name,statusEN,LMR,familyEN,typeEN
pesticide,,,,,
Aclonifen,aclonifen,authorized,0.05,"diphenyl,ether",herbicide
Aldrin,aldrin,prohibited,0.01,organochlorine,insecticide
Ametoctradin,ametoctradin,authorized,0.05,triazolopyrimidine,fungicide
Anthraquinone,anthraquinone,prohibited,0.02,"polycyclic,aromatic,hydrocarbon",birdrepellant
Azoxystrobin,azoxystrobin,authorized,0.05,strobilurin,fungicide
Benalaxyl,benalaxyl,prohibited,0.05,phenylamides,fungicide
Benfluralin,benfluralin,authorized,0.05,dinitroaniline,herbicide
Benthiavalicarb.iso,benthiavalicarb-isopropyl,authorized,0.05,carbamate,fungicide
Benzovindiflupyr,benzovindiflupyr,authorized,0.05,amide,fungicide
